In [ ]:
import xarray as xr         # For handling NetCDF data
import numpy as np          # For array operations
import geopandas as gpd     # For geospatial dataframes and file I/O
from shapely.geometry import Point  # For creating point geometries
import xwrf

In [ ]:
# get WRF urban mask
wrfout_sample = '/project2/zhan248_1326/hhao4018/WRFv4.6.1_Modified_AHoption_noChem/test/JOE_Default_2016_08_ah=0/wrfout_d02_2016-08-12_20:00:00'
ds_temp = xr.open_dataset(wrfout_sample).xwrf.postprocess()
frc_urb2d = ds_temp['FRC_URB2D'][0]
wrf_urban_mask = (frc_urb2d > 0)

# save as geopackage
# Get lat/lon coordinates
lat = ds_temp['XLAT']
lon = ds_temp['XLONG']

# Flatten arrays and filter for urban cells
urban_lats = lat.values[wrf_urban_mask.values]
urban_lons = lon.values[wrf_urban_mask.values]

# Create GeoDataFrame of urban points
gdf = gpd.GeoDataFrame(
    geometry=[Point(lon, lat) for lon, lat in zip(urban_lons, urban_lats)],
    crs="EPSG:4326"
)

# Save to file (GeoPackage recommended)
gdf.to_file("/home1/kojoseph/ah-la-paper/data/wrf_urban_mask.gpkg", driver="GPKG")